In [1]:
import matplotlib.pyplot as plt
import glob
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

tf.config.list_physical_devices('GPU')

#データ準備
path = "train-data/"

files = glob.glob(path + "train/*.jpg")   #.jpgのパスを取得
files = sorted(files)
df_label = pd.read_csv(path + "train.csv")

# file_pathとcsv行を抽出する
def inexclusion(num: int):
  return files[:num], df_label.head(num)

files, df_label = inexclusion(2500)

file_list = []
for file in files:
  file = cv2.imread(file)
  file_list.append(file)

#要素数の確認
print(len(file_list),len(df_label))

2023-12-22 05:43:34.435934: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-22 05:43:34.463347: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-22 05:43:34.463362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-22 05:43:34.464019: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-22 05:43:34.468910: I tensorflow/core/platform/cpu_feature_guar

2500 2500


In [2]:
#画素値を正規化
file_list = [file.astype(float)/255 for file in file_list]
train_x, valid_x, train_y, valid_y = train_test_split(file_list, df_label, test_size=0.2)

# train_y, valid_y をダミー変数化
train_y = to_categorical(train_y["gender_status"])
valid_y = to_categorical(valid_y["gender_status"])

# リスト型を配列型に
train_x = np.array(train_x)
valid_x = np.array(valid_x)

In [3]:
#層の定義
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))

# モデルを構築
model.compile(optimizer=tf.optimizers.Adam(0.01), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stoppingを適用してフィッティング
log = model.fit(train_x, train_y, epochs=100, batch_size=10, verbose=True,
                callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                     min_delta=0, patience=10,
                                                         verbose=1)],
                validation_data=(valid_x, valid_y))

loss,accuracy = model.evaluate(valid_x,valid_y,verbose = 0)
print('Accuracy','{:.2f}'.format(accuracy))

2023-12-22 05:43:39.322695: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-22 05:43:39.322838: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-22 05:43:39.322865: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-22 05:43:39.419842: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-22 05:43:39.419903: I external/local_xla/xla/stream_executor

Epoch 1/100


2023-12-22 05:43:42.087243: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-22 05:43:42.443138: I external/local_xla/xla/service/service.cc:168] XLA service 0x1512abe62750 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-22 05:43:42.443164: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2023-12-22 05:43:42.447661: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1703223822.525871   14603 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


200/200 [==============================] - 6s 17ms/step - loss: 7.0229 - accuracy: 0.2855 - val_loss: 1.6249 - val_accuracy: 0.3020
Epoch 2/100
200/200 [==============================] - 3s 15ms/step - loss: 1.6236 - accuracy: 0.2855 - val_loss: 1.6232 - val_accuracy: 0.3020
Epoch 3/100
200/200 [==============================] - 3s 14ms/step - loss: 1.6238 - accuracy: 0.2840 - val_loss: 1.6248 - val_accuracy: 0.2720
Epoch 4/100
200/200 [==============================] - 3s 14ms/step - loss: 1.6248 - accuracy: 0.2850 - val_loss: 1.6268 - val_accuracy: 0.3020
Epoch 5/100
200/200 [==============================] - 3s 15ms/step - loss: 1.6218 - accuracy: 0.2970 - val_loss: 1.6243 - val_accuracy: 0.3020
Epoch 6/100
200/200 [==============================] - 3s 14ms/step - loss: 1.6238 - accuracy: 0.2955 - val_loss: 1.6205 - val_accuracy: 0.2720
Epoch 7/100
200/200 [==============================] - 3s 14ms/step - loss: 1.6238 - accuracy: 0.2915 - val_loss: 1.6227 - val_accuracy: 0.3020
Epoc